In [4]:
import numpy as np
import os
import torch
import torch.nn as nn 
from datetime import datetime
from sklearn import preprocessing 
import pickle 
import scipy.io as sio 
from TraditionalFusion.utils import generating_data, svm_classification
import time 


from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
import pandas as pd

In [5]:
os.environ['FILE_PATH'] = '../SEED/SEED_EEG/'
FILE_PATH = os.getenv('FILE_PATH', '/mnt/g/Meu Drive/mestrado_ppget')
print(FILE_PATH)
eeg_dir = os.path.join(FILE_PATH,"SEED_EEG/ExtractedFeatures/")
eeg_file_list = os.listdir(eeg_dir)
eeg_file_list.sort()


feature_list = ['de_LDS']


../SEED/SEED_EEG/


In [6]:
print(eeg_file_list)

['10_20131130.mat', '10_20131204.mat', '10_20131211.mat', '11_20140618.mat', '11_20140625.mat', '11_20140630.mat', '12_20131127.mat', '12_20131201.mat', '12_20131207.mat', '13_20140527.mat', '13_20140603.mat', '13_20140610.mat', '14_20140601.mat', '14_20140615.mat', '14_20140627.mat', '15_20130709.mat', '15_20131016.mat', '15_20131105.mat', '1_20131027.mat', '1_20131030.mat', '1_20131107.mat', '2_20140404.mat', '2_20140413.mat', '2_20140419.mat', '3_20140603.mat', '3_20140611.mat', '3_20140629.mat', '4_20140621.mat', '4_20140702.mat', '4_20140705.mat', '5_20140411.mat', '5_20140418.mat', '5_20140506.mat', '6_20130712.mat', '6_20131016.mat', '6_20131113.mat', '7_20131027.mat', '7_20131030.mat', '7_20131106.mat', '8_20140511.mat', '8_20140514.mat', '8_20140521.mat', '9_20140620.mat', '9_20140627.mat', '9_20140704.mat', 'label.mat', 'readme.txt']


In [7]:
item = eeg_file_list[0]
fn = feature_list[0]
label_list = np.array([2,1,0,0,1,2,0,1,2,2,1,0,1,2,0]) 


cur_user = item.split("_")[0]
user_session = datetime.strptime(item.split("_")[1][:-4],"%Y%m%d")
print(f"- reading file: {item}\n--user:{cur_user}, sessions: {user_session}")


- reading file: 10_20131130.mat
--user:10, sessions: 2013-11-30 00:00:00


In [8]:
all_data = sio.loadmat(os.path.join(eeg_dir, item))

In [ ]:
def generating_data(data_dict, clip_label, feature_name):
    # first 9 movies as training, the last 6 movies as testing
    train_data = data_dict[feature_name+'1']
    print("\t### TRAIN ###")
    print(f"\t\t{feature_name + '1'} - video 1 - {train_data.shape} observacoes")
    _, num, _ = train_data.shape 
    train_label = np.zeros(num,) + clip_label[0]
    train_data = np.swapaxes(train_data, 0, 1)
    train_data = np.reshape(train_data, (num, -1))
    train_residual_index = [2,3,4,5,6,7,8,9]
    for ind,i in enumerate(train_residual_index):
        used_data = data_dict[feature_name + str(i)]
        _, num, _ = used_data.shape 
        used_label = np.zeros(num,) + clip_label[ind+1]
        used_data = np.swapaxes(used_data, 0, 1)
        used_data = np.reshape(used_data, (num, -1))
        print(f"\t\t{feature_name + str(i)} - video {str(i)} - {used_data.shape} observacoes")
        train_data = np.vstack((train_data, used_data))
        train_label = np.hstack((train_label, used_label))
    
    test_data = data_dict[feature_name+'10']
    print("\t### TEST ###")
    print(f"\t\t{feature_name + '10'} - video 10 - {test_data.shape} observacoes")
    _, num, _ = test_data.shape 
    test_label = np.zeros(num,) + clip_label[9]
    test_data = np.swapaxes(test_data, 0, 1)
    test_data = np.reshape(test_data, (num, -1))
    test_residual_index = [11,12,13,14,15]
    for ind,i in enumerate(test_residual_index):
        used_data = data_dict[feature_name + str(i)]
        _, num, _ = used_data.shape 
        used_label = np.zeros(num,) + clip_label[ind+9]
        used_data = np.swapaxes(used_data, 0, 1)
        used_data = np.reshape(used_data, (num, -1))
        print(f"\t\t{feature_name + str(i)} - video {str(i)} - {used_data.shape} observacoes")
        test_data = np.vstack((test_data, used_data))
        test_label = np.hstack((test_label, used_label))
    return train_data, test_data, train_label, test_label

In [ ]:
train_data, test_data, train_label, test_label = generating_data(all_data, label_list, fn)


In [ ]:
train_data.shape

In [ ]:
train_data.shape

verificando com dado original para ver se faz sentido

In [ ]:
all_data[fn+'1']

In [ ]:
data_to_validate = all_data[fn+'1']

print(data_to_validate.shape)
EEG_CHANEL = 0
print(data_to_validate[EEG_CHANEL].shape)
data_to_validate[EEG_CHANEL]


In [ ]:
all_eeg_channel, num, bands = data_to_validate.shape 

data_to_validate_1 = np.swapaxes(data_to_validate, 0, 1)
data_to_validate_1.shape

In [ ]:
data_to_validate_1[0]

In [ ]:
data_to_validate_2 = np.reshape(data_to_validate_1, (num, -1))
data_to_validate_2.shape

In [ ]:
data_to_validate_2[0]

aparentemente tudo ok!

# Treinando SVM

In [ ]:
print(item)
c = 2

clf = svm.LinearSVC(C=2**c)
clf.fit(train_data, train_label)    
p_labels = clf.predict(test_data)


from sklearn import metrics
score = clf.score(test_data, test_label)
print(f"SCORE: {score}")
print("Accuracy:",metrics.accuracy_score(test_label, p_labels))
# print("Precision:",metrics.precision_score(test_label, p_labels))
# print("Recall:",metrics.recall_score(test_label, p_labels))





In [ ]:
test_data.shape

In [ ]:
test_label[472:705]

In [ ]:
p_labels[472:705]

# Gerando dados com dataframe

In [6]:
data_dict, clip_label, feature_name = all_data, label_list, fn

## treino

In [29]:
# def generating_data(data_dict, clip_label, feature_name):
# first 9 movies as training, the last 6 movies as testing
train_data = data_dict[feature_name+'1']
print("\t### TRAIN ###")
print(f"\t\t{feature_name + '1'} - video 1 - {train_data.shape} observacoes")
_, num, _ = train_data.shape
used_train_film_ref = np.zeros(num,) + 1
train_label = np.zeros(num,) + clip_label[0]
train_data = np.swapaxes(train_data, 0, 1)
train_data

	### TRAIN ###
		de_LDS1 - video 1 - (62, 235, 5) observacoes


array([[[27.08358157, 22.91154056, 21.15346081, 19.74299538,
         19.15590487],
        [27.17584979, 22.94684656, 21.14874759, 18.96109104,
         17.88738954],
        [26.9454401 , 22.86094665, 21.17235922, 20.20211872,
         19.77274198],
        ...,
        [26.31324229, 21.99120193, 21.05632946, 19.37831684,
         18.51801561],
        [26.51090791, 22.05971002, 20.98619159, 19.28105985,
         18.17318956],
        [26.63291956, 22.09148031, 20.9950124 , 19.32675924,
         18.13914969]],

       [[27.08847498, 22.90886303, 21.15102226, 19.74405133,
         19.15741419],
        [27.17383851, 22.9448068 , 21.14691101, 18.96060752,
         17.88768093],
        [26.94605675, 22.85896687, 21.17035765, 20.20284772,
         19.77301064],
        ...,
        [26.31367824, 21.99143826, 21.05753684, 19.37912317,
         18.51905356],
        [26.51437227, 22.06071828, 20.98728973, 19.28197287,
         18.17383787],
        [26.63675038, 22.09250785, 20.99591001, 

In [30]:
train_data = np.reshape(train_data, (num, -1))
train_data

array([[27.08358157, 22.91154056, 21.15346081, ..., 20.9950124 ,
        19.32675924, 18.13914969],
       [27.08847498, 22.90886303, 21.15102226, ..., 20.99591001,
        19.32718201, 18.13893663],
       [27.09624093, 22.90799629, 21.15015659, ..., 20.99704567,
        19.32747367, 18.13900468],
       ...,
       [26.3270917 , 22.77351568, 21.02755558, ..., 20.76392779,
        19.35880254, 18.33475183],
       [26.32816947, 22.77516187, 21.02765953, ..., 20.76293518,
        19.35824372, 18.33460931],
       [26.32750318, 22.77408488, 21.02622243, ..., 20.7618358 ,
        19.35745408, 18.33352827]])

In [23]:
train_data.shape

(235, 310)

In [53]:
used_data = data_dict[feature_name + str(i)]
used_data.shape

(62, 206, 5)

In [31]:
train_residual_index = [2,3,4,5,6,7,8,9]
for ind,i in enumerate(train_residual_index):
    used_data = data_dict[feature_name + str(i)]
    _, num, _ = used_data.shape 
    used_label = np.zeros(num,) + clip_label[ind+1]

    used_train_film_ref = np.zeros(num,) + i #vai gerar um vetor com a referencia do filme. Ex: (235 primeiros valores referem-se ao filme 1)

    used_data = np.swapaxes(used_data, 0, 1)
    used_data = np.reshape(used_data, (num, -1))
    print(f"\t\t{feature_name + str(i)} - video {str(i)} - {used_data.shape} observacoes")
    train_data = np.vstack((train_data, used_data))

    train_label = np.hstack((train_label, used_label))

		de_LDS2 - video 2 - (233, 310) observacoes
		de_LDS3 - video 3 - (206, 310) observacoes
		de_LDS4 - video 4 - (238, 310) observacoes
		de_LDS5 - video 5 - (185, 310) observacoes
		de_LDS6 - video 6 - (195, 310) observacoes
		de_LDS7 - video 7 - (237, 310) observacoes
		de_LDS8 - video 8 - (216, 310) observacoes
		de_LDS9 - video 9 - (265, 310) observacoes


array([[27.08358157, 22.91154056, 21.15346081, ..., 20.9950124 ,
        19.32675924, 18.13914969],
       [27.08847498, 22.90886303, 21.15102226, ..., 20.99591001,
        19.32718201, 18.13893663],
       [27.09624093, 22.90799629, 21.15015659, ..., 20.99704567,
        19.32747367, 18.13900468],
       ...,
       [26.46839952, 22.68214713, 21.12216414, ..., 20.80866727,
        19.17956115, 18.15544733],
       [26.46748892, 22.68504015, 21.12516397, ..., 20.80912973,
        19.17944841, 18.1553233 ],
       [26.46881893, 22.68545062, 21.12657624, ..., 20.80823875,
        19.17961697, 18.15547013]])

In [44]:
pd.DataFrame(train_data)

,0,1,2,3,4,5,6,7,8,9,...,300,301,302,303,304,305,306,307,308,309
0,27.083582,22.911541,21.153461,19.742995,19.155905,27.175850,22.946847,21.148748,18.961091,17.887390,...,26.510908,22.059710,20.986192,19.281060,18.173190,26.632920,22.091480,20.995012,19.326759,18.139150
1,27.088475,22.908863,21.151022,19.744051,19.157414,27.173839,22.944807,21.146911,18.960608,17.887681,...,26.514372,22.060718,20.987290,19.281973,18.173838,26.636750,22.092508,20.995910,19.327182,18.138937
2,27.096241,22.907996,21.150157,19.745564,19.158397,27.172756,22.944278,21.146201,18.960896,17.887408,...,26.519048,22.062376,20.988591,19.282761,18.174713,26.642167,22.094087,20.997046,19.327474,18.139005
3,27.106098,22.906658,21.149065,19.747945,19.159047,27.172432,22.943135,21.145391,18.961505,17.886351,...,26.523506,22.063644,20.989143,19.283235,18.175773,26.647310,22.095252,20.997387,19.327520,18.139236
4,27.113424,22.902107,21.145085,19.749826,19.159116,27.169961,22.939050,21.141944,18.961401,17.885342,...,26.527516,22.067019,20.989148,19.283450,18.177211,26.651835,22.098452,20.997137,19.327295,18.139559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005,26.465537,22.679560,21.115567,19.387337,18.892025,26.388043,22.759533,21.121713,18.716031,17.693382,...,25.377162,21.864965,20.687678,19.007929,17.992723,25.456353,21.972640,20.807338,19.180774,18.153958
2006,26.468310,22.679707,21.118323,19.387951,18.891181,26.388913,22.759646,21.124569,18.716567,17.692947,...,25.374407,21.864644,20.687298,19.007728,17.992952,25.452963,21.972395,20.807518,19.180046,18.154838
2007,26.468400,22.682147,21.122164,19.389112,18.891033,26.387655,22.762193,21.128242,18.717983,17.692667,...,25.376399,21.866243,20.688373,19.007473,17.993450,25.455270,21.973992,20.808667,19.179561,18.155447
2008,26.467489,22.685040,21.125164,19.390889,18.890967,26.385324,22.765207,21.131105,18.719317,17.692512,...,25.379142,21.866695,20.688766,19.007554,17.993816,25.458271,21.974444,20.809130,19.179448,18.155323


## teste

In [45]:
test_data = data_dict[feature_name+'10']
print("\t### TEST ###")
print(f"\t\t{feature_name + '10'} - video 10 - {test_data.shape} observacoes")
_, num, _ = test_data.shape 
test_label = np.zeros(num,) + clip_label[9]
test_data = np.swapaxes(test_data, 0, 1)
test_data = np.reshape(test_data, (num, -1))
test_residual_index = [11,12,13,14,15]
for ind,i in enumerate(test_residual_index):
    used_data = data_dict[feature_name + str(i)]
    _, num, _ = used_data.shape 
    used_label = np.zeros(num,) + clip_label[ind+10]
    used_data = np.swapaxes(used_data, 0, 1)
    used_data = np.reshape(used_data, (num, -1))
    print(f"\t\t{feature_name + str(i)} - video {str(i)} - {used_data.shape} observacoes")
    test_data = np.vstack((test_data, used_data))
    test_label = np.hstack((test_label, used_label))

	### TEST ###
		de_LDS10 - video 10 - (62, 237, 5) observacoes
		de_LDS11 - video 11 - (235, 310) observacoes
		de_LDS12 - video 12 - (233, 310) observacoes
		de_LDS13 - video 13 - (235, 310) observacoes
		de_LDS14 - video 14 - (238, 310) observacoes
		de_LDS15 - video 15 - (206, 310) observacoes


In [49]:
all_data = np.vstack((train_data,test_data))
pd.DataFrame(all_data)

,0,1,2,3,4,5,6,7,8,9,...,300,301,302,303,304,305,306,307,308,309
0,27.083582,22.911541,21.153461,19.742995,19.155905,27.175850,22.946847,21.148748,18.961091,17.887390,...,26.510908,22.059710,20.986192,19.281060,18.173190,26.632920,22.091480,20.995012,19.326759,18.139150
1,27.088475,22.908863,21.151022,19.744051,19.157414,27.173839,22.944807,21.146911,18.960608,17.887681,...,26.514372,22.060718,20.987290,19.281973,18.173838,26.636750,22.092508,20.995910,19.327182,18.138937
2,27.096241,22.907996,21.150157,19.745564,19.158397,27.172756,22.944278,21.146201,18.960896,17.887408,...,26.519048,22.062376,20.988591,19.282761,18.174713,26.642167,22.094087,20.997046,19.327474,18.139005
3,27.106098,22.906658,21.149065,19.747945,19.159047,27.172432,22.943135,21.145391,18.961505,17.886351,...,26.523506,22.063644,20.989143,19.283235,18.175773,26.647310,22.095252,20.997387,19.327520,18.139236
4,27.113424,22.902107,21.145085,19.749826,19.159116,27.169961,22.939050,21.141944,18.961401,17.885342,...,26.527516,22.067019,20.989148,19.283450,18.177211,26.651835,22.098452,20.997137,19.327295,18.139559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3389,27.911546,23.531114,21.620189,19.953264,19.829079,28.092264,23.547074,21.563025,19.292745,19.048496,...,26.932187,21.539609,20.659480,19.179487,18.355424,26.955946,21.645534,20.691226,19.299889,18.277825
3390,27.907846,23.525218,21.616800,19.953934,19.829158,28.090265,23.541538,21.560456,19.292059,19.048796,...,26.925897,21.533209,20.659789,19.181338,18.357258,26.947196,21.639681,20.689758,19.301448,18.279000
3391,27.902236,23.521841,21.615662,19.954796,19.829844,28.087716,23.538201,21.559766,19.291783,19.048619,...,26.921329,21.526777,20.659073,19.182048,18.357918,26.939745,21.634083,20.687578,19.302417,18.279012
3392,27.897143,23.519366,21.614309,19.955873,19.830709,28.087133,23.535592,21.558888,19.291820,19.049158,...,26.919992,21.521520,20.658873,19.182985,18.357885,26.936840,21.629278,20.686986,19.303100,18.278745


In [ ]:
#TODO adicionar a label de cada filme

# TESTE DE FUNCAO

In [3]:
import numpy as np
import os
import torch
import torch.nn as nn 
from datetime import datetime
from sklearn import preprocessing 
import pickle 
import scipy.io as sio 
from TraditionalFusion.utils import generating_data, svm_classification
import time 


from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
import pandas as pd

In [4]:
os.environ['FILE_PATH'] = '../SEED/SEED_EEG/'
FILE_PATH = os.getenv('FILE_PATH', '/mnt/g/Meu Drive/mestrado_ppget')
print(FILE_PATH)
eeg_dir = os.path.join(FILE_PATH,"SEED_EEG/ExtractedFeatures/")
eeg_file_list = os.listdir(eeg_dir)
eeg_file_list.sort()


feature_list = ['de_LDS']


../SEED/SEED_EEG/


In [15]:
item = eeg_file_list[0]
fn = feature_list[0]
label_list = np.array([2,1,0,0,1,2,0,1,2,2,1,0,1,2,0]) 


cur_user = item.split("_")[0]
user_session = datetime.strptime(item.split("_")[1][:-4],"%Y%m%d")
print(f"- reading file: {item}\n--user:{cur_user}, sessions: {user_session}")


- reading file: 10_20131130.mat
--user:10, sessions: 2013-11-30 00:00:00


In [16]:
all_data = sio.loadmat(os.path.join(eeg_dir, item))

In [5]:
channel_order = pd.read_excel(os.path.join(FILE_PATH, "SEED_EEG/channel-order.xlsx"), header=None).reset_index()
# channel_order['index'] = channel_order.index+1
print(channel_order)
# channel_order[channel_order['index'] == eeg_62channels_idx][0].values[0]
print(channel_order[0].to_list())

    index    0
0       0  FP1
1       1  FPZ
2       2  FP2
3       3  AF3
4       4  AF4
..    ...  ...
57     57  CB1
58     58   O1
59     59   OZ
60     60   O2
61     61  CB2

[62 rows x 2 columns]
['FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', 'CB1', 'O1', 'OZ', 'O2', 'CB2']


In [6]:
all_freqs = ["delta", "theta", "alpha", "beta", "gamma"]

In [7]:
import itertools

all_channel_freq = list(itertools.product(channel_order[0].to_list(), all_freqs))
print(all_channel_freq)

[('FP1', 'delta'), ('FP1', 'theta'), ('FP1', 'alpha'), ('FP1', 'beta'), ('FP1', 'gamma'), ('FPZ', 'delta'), ('FPZ', 'theta'), ('FPZ', 'alpha'), ('FPZ', 'beta'), ('FPZ', 'gamma'), ('FP2', 'delta'), ('FP2', 'theta'), ('FP2', 'alpha'), ('FP2', 'beta'), ('FP2', 'gamma'), ('AF3', 'delta'), ('AF3', 'theta'), ('AF3', 'alpha'), ('AF3', 'beta'), ('AF3', 'gamma'), ('AF4', 'delta'), ('AF4', 'theta'), ('AF4', 'alpha'), ('AF4', 'beta'), ('AF4', 'gamma'), ('F7', 'delta'), ('F7', 'theta'), ('F7', 'alpha'), ('F7', 'beta'), ('F7', 'gamma'), ('F5', 'delta'), ('F5', 'theta'), ('F5', 'alpha'), ('F5', 'beta'), ('F5', 'gamma'), ('F3', 'delta'), ('F3', 'theta'), ('F3', 'alpha'), ('F3', 'beta'), ('F3', 'gamma'), ('F1', 'delta'), ('F1', 'theta'), ('F1', 'alpha'), ('F1', 'beta'), ('F1', 'gamma'), ('FZ', 'delta'), ('FZ', 'theta'), ('FZ', 'alpha'), ('FZ', 'beta'), ('FZ', 'gamma'), ('F2', 'delta'), ('F2', 'theta'), ('F2', 'alpha'), ('F2', 'beta'), ('F2', 'gamma'), ('F4', 'delta'), ('F4', 'theta'), ('F4', 'alpha'),

In [21]:
all_channel_freq_as_column = [each_ch_fr[0]+"_"+each_ch_fr[1] for each_ch_fr in all_channel_freq]
print(all_channel_freq_as_column)

['FP1_delta', 'FP1_theta', 'FP1_alpha', 'FP1_beta', 'FP1_gamma', 'FPZ_delta', 'FPZ_theta', 'FPZ_alpha', 'FPZ_beta', 'FPZ_gamma', 'FP2_delta', 'FP2_theta', 'FP2_alpha', 'FP2_beta', 'FP2_gamma', 'AF3_delta', 'AF3_theta', 'AF3_alpha', 'AF3_beta', 'AF3_gamma', 'AF4_delta', 'AF4_theta', 'AF4_alpha', 'AF4_beta', 'AF4_gamma', 'F7_delta', 'F7_theta', 'F7_alpha', 'F7_beta', 'F7_gamma', 'F5_delta', 'F5_theta', 'F5_alpha', 'F5_beta', 'F5_gamma', 'F3_delta', 'F3_theta', 'F3_alpha', 'F3_beta', 'F3_gamma', 'F1_delta', 'F1_theta', 'F1_alpha', 'F1_beta', 'F1_gamma', 'FZ_delta', 'FZ_theta', 'FZ_alpha', 'FZ_beta', 'FZ_gamma', 'F2_delta', 'F2_theta', 'F2_alpha', 'F2_beta', 'F2_gamma', 'F4_delta', 'F4_theta', 'F4_alpha', 'F4_beta', 'F4_gamma', 'F6_delta', 'F6_theta', 'F6_alpha', 'F6_beta', 'F6_gamma', 'F8_delta', 'F8_theta', 'F8_alpha', 'F8_beta', 'F8_gamma', 'FT7_delta', 'FT7_theta', 'FT7_alpha', 'FT7_beta', 'FT7_gamma', 'FC5_delta', 'FC5_theta', 'FC5_alpha', 'FC5_beta', 'FC5_gamma', 'FC3_delta', 'FC3_th

In [13]:
def generating_data_as_dataframe(data_dict, clip_label, feature_name):
    # first 9 movies as training, the last 6 movies as testing
    train_data = data_dict[feature_name+'1']
    print("\t### TRAIN ###")
    print(f"\t\t{feature_name + '1'} - video 1 - {train_data.shape} observacoes")
    _, num, _ = train_data.shape 
    train_label = np.zeros(num,) + clip_label[0]

    train_film_ref = np.zeros(num,) + 1

    train_data = np.swapaxes(train_data, 0, 1)
    train_data = np.reshape(train_data, (num, -1))
    train_residual_index = [2,3,4,5,6,7,8,9]
    for ind,i in enumerate(train_residual_index):
        used_data = data_dict[feature_name + str(i)]
        _, num, _ = used_data.shape 
        used_label = np.zeros(num,) + clip_label[ind+1]

        used_film_ref = np.zeros(num,) + i

        used_data = np.swapaxes(used_data, 0, 1)
        used_data = np.reshape(used_data, (num, -1))
        print(f"\t\t{feature_name + str(i)} - video {str(i)} - {used_data.shape} observacoes")
        train_data = np.vstack((train_data, used_data))
        train_label = np.hstack((train_label, used_label))

        train_film_ref = np.hstack((train_film_ref, used_film_ref))
     
    test_data = data_dict[feature_name+'10']
    print("\t### TEST ###")
    print(f"\t\t{feature_name + '10'} - video 10 - {test_data.shape} observacoes")
    _, num, _ = test_data.shape 
    test_label = np.zeros(num,) + clip_label[9]

    test_film_ref = np.zeros(num,) + 10

    test_data = np.swapaxes(test_data, 0, 1)
    test_data = np.reshape(test_data, (num, -1))
    test_residual_index = [11,12,13,14,15]
    for ind,i in enumerate(test_residual_index):
        used_data = data_dict[feature_name + str(i)]
        _, num, _ = used_data.shape 
        used_label = np.zeros(num,) + clip_label[ind+10]

        used_film_ref = np.zeros(num,) + i

        used_data = np.swapaxes(used_data, 0, 1)
        used_data = np.reshape(used_data, (num, -1))
        print(f"\t\t{feature_name + str(i)} - video {str(i)} - {used_data.shape} observacoes")
        test_data = np.vstack((test_data, used_data))
        test_label = np.hstack((test_label, used_label))

        test_film_ref = np.hstack((test_film_ref, used_film_ref))

    all_data = np.vstack((train_data,test_data))
    all_data_df = pd.DataFrame(all_data)

    all_data_labels = np.hstack((train_label, test_label))

    all_films_ref = np.hstack((train_film_ref, test_film_ref))
    all_films_ref_df = pd.DataFrame(all_films_ref)
    return all_data_df, all_data_labels, all_films_ref_df

In [17]:
all_EEG_data_df, all_films_labels,all_films_ref = generating_data_as_dataframe(all_data, label_list, fn)


	### TRAIN ###
		de_LDS1 - video 1 - (62, 235, 5) observacoes
		de_LDS2 - video 2 - (233, 310) observacoes
		de_LDS3 - video 3 - (206, 310) observacoes
		de_LDS4 - video 4 - (238, 310) observacoes
		de_LDS5 - video 5 - (185, 310) observacoes
		de_LDS6 - video 6 - (195, 310) observacoes
		de_LDS7 - video 7 - (237, 310) observacoes
		de_LDS8 - video 8 - (216, 310) observacoes
		de_LDS9 - video 9 - (265, 310) observacoes
	### TEST ###
		de_LDS10 - video 10 - (62, 237, 5) observacoes
		de_LDS11 - video 11 - (235, 310) observacoes
		de_LDS12 - video 12 - (233, 310) observacoes
		de_LDS13 - video 13 - (235, 310) observacoes
		de_LDS14 - video 14 - (238, 310) observacoes
		de_LDS15 - video 15 - (206, 310) observacoes


In [18]:
all_films_ref

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
3389,15.0
3390,15.0
3391,15.0
3392,15.0


In [19]:
all_EEG_data_df['film_ref'] = all_films_ref
all_EEG_data_df['all_films_labels'] = all_films_labels
all_EEG_data_df

,0,1,2,3,4,5,6,7,8,9,...,302,303,304,305,306,307,308,309,film_ref,all_films_labels
0,27.083582,22.911541,21.153461,19.742995,19.155905,27.175850,22.946847,21.148748,18.961091,17.887390,...,20.986192,19.281060,18.173190,26.632920,22.091480,20.995012,19.326759,18.139150,1.0,2.0
1,27.088475,22.908863,21.151022,19.744051,19.157414,27.173839,22.944807,21.146911,18.960608,17.887681,...,20.987290,19.281973,18.173838,26.636750,22.092508,20.995910,19.327182,18.138937,1.0,2.0
2,27.096241,22.907996,21.150157,19.745564,19.158397,27.172756,22.944278,21.146201,18.960896,17.887408,...,20.988591,19.282761,18.174713,26.642167,22.094087,20.997046,19.327474,18.139005,1.0,2.0
3,27.106098,22.906658,21.149065,19.747945,19.159047,27.172432,22.943135,21.145391,18.961505,17.886351,...,20.989143,19.283235,18.175773,26.647310,22.095252,20.997387,19.327520,18.139236,1.0,2.0
4,27.113424,22.902107,21.145085,19.749826,19.159116,27.169961,22.939050,21.141944,18.961401,17.885342,...,20.989148,19.283450,18.177211,26.651835,22.098452,20.997137,19.327295,18.139559,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3389,27.911546,23.531114,21.620189,19.953264,19.829079,28.092264,23.547074,21.563025,19.292745,19.048496,...,20.659480,19.179487,18.355424,26.955946,21.645534,20.691226,19.299889,18.277825,15.0,0.0
3390,27.907846,23.525218,21.616800,19.953934,19.829158,28.090265,23.541538,21.560456,19.292059,19.048796,...,20.659789,19.181338,18.357258,26.947196,21.639681,20.689758,19.301448,18.279000,15.0,0.0
3391,27.902236,23.521841,21.615662,19.954796,19.829844,28.087716,23.538201,21.559766,19.291783,19.048619,...,20.659073,19.182048,18.357918,26.939745,21.634083,20.687578,19.302417,18.279012,15.0,0.0
3392,27.897143,23.519366,21.614309,19.955873,19.830709,28.087133,23.535592,21.558888,19.291820,19.049158,...,20.658873,19.182985,18.357885,26.936840,21.629278,20.686986,19.303100,18.278745,15.0,0.0


In [22]:
all_EEG_data_df.columns = ([*all_channel_freq_as_column, 'film_ref', 'all_films_labels'])

In [23]:
session_esc = user_session.strftime("%Y%m%d")
all_EEG_data_df.to_csv(f"{cur_user}_{session_esc}.csv")

In [24]:
user_session.strftime("%Y%m%d")

'20131130'

# SEPARAR FREQUENCIAS

In [41]:
frequencies_selection=['alpha']
all_freqs_dict = {
    "delta":0,
    "theta":1,
    "alpha":2,
    "beta":3,
    "gamma":4}

In [44]:
data_dict, clip_label, feature_name = all_data, label_list, fn

In [63]:
train_data = data_dict[feature_name+'1']
print("\t### TRAIN ###")
print(f"\t\t{feature_name + '1'} - video 1 - {train_data.shape} observacoes")
_, num, _ = train_data.shape 
train_label = np.zeros(num,) + clip_label[0]


	### TRAIN ###
		de_LDS1 - video 1 - (62, 235, 5) observacoes


In [64]:
train_data.shape # vou selecionar apenas da ultima parte do vetor (5)

(62, 235, 5)

In [65]:
train_data = np.swapaxes(train_data, 0, 1)


In [66]:
train_data.shape

(235, 62, 5)

In [67]:
if 'all' in frequencies_selection:
    train_data = np.reshape(train_data, (num, -1))
else:
    # apenas uma frequencia por vez
    # TODO processamento paralelo de cada frequencia
    frequencie_idx = all_freqs_dict[frequencies_selection[0]]
    print(f"## select only {frequencies_selection} freq - idx: {frequencie_idx} ##")
    train_data = train_data[:,:,frequencie_idx]

train_data.shape


## select only ['alpha'] freq - idx: 2


(235, 62)

In [33]:
type(train_data)


numpy.ndarray

In [43]:
train_data[:,:,4].shape

(235, 62)

## teste de funcao

In [86]:
# from TraditionalFusion.utils import generating_data
# frequencies_selection=['alpha']
all_freqs_dict = {
    "delta":0,
    "theta":1,
    "alpha":2,
    "beta":3,
    "gamma":4}

def generating_data(data_dict, clip_label, feature_name, frequencies_selection=['all']):
    # first 9 movies as training, the last 6 movies as testing
    train_data = data_dict[feature_name+'1']
    print("\t### TRAIN ###")
    _, num, _ = train_data.shape 
    train_label = np.zeros(num,) + clip_label[0]
    train_data = np.swapaxes(train_data, 0, 1)
    if 'all' in frequencies_selection:
        train_data = np.reshape(train_data, (num, -1))
    else:
        # apenas uma frequencia por vez
        # TODO processamento paralelo de cada frequencia
        frequencie_idx = all_freqs_dict[frequencies_selection[0]]
        print(f"## select only {frequencies_selection} freq - idx: {frequencie_idx} ##")
        train_data = train_data[:,:,frequencie_idx]
    print(f"\t\t{feature_name + '1'} - video 1 - {train_data.shape} observacoes")
    train_residual_index = [2,3,4,5,6,7,8,9]
    for ind,i in enumerate(train_residual_index):
        used_data = data_dict[feature_name + str(i)]
        _, num, _ = used_data.shape 
        used_label = np.zeros(num,) + clip_label[ind+1]
        used_data = np.swapaxes(used_data, 0, 1)
        if 'all' in frequencies_selection:
            used_data = np.reshape(used_data, (num, -1))
        else:
            # apenas uma frequencia por vez
            # TODO processamento paralelo de cada frequencia
            frequencie_idx = all_freqs_dict[frequencies_selection[0]]
            used_data = used_data[:,:,frequencie_idx]
            
        print(f"\t\t{feature_name + str(i)} - video {str(i)} - {used_data.shape} observacoes")
        train_data = np.vstack((train_data, used_data))
        train_label = np.hstack((train_label, used_label))
    
    test_data = data_dict[feature_name+'10']
    print("\t### TEST ###")
    _, num, _ = test_data.shape 
    test_label = np.zeros(num,) + clip_label[9]
    test_data = np.swapaxes(test_data, 0, 1)
    if 'all' in frequencies_selection:
        test_data = np.reshape(test_data, (num, -1))
    else:
        # apenas uma frequencia por vez
        # TODO processamento paralelo de cada frequencia
        frequencie_idx = all_freqs_dict[frequencies_selection[0]]
        test_data = test_data[:,:,frequencie_idx]
    print(f"\t\t{feature_name + '10'} - video 10 - {test_data.shape} observacoes")
    test_residual_index = [11,12,13,14,15]
    for ind,i in enumerate(test_residual_index):
        used_data = data_dict[feature_name + str(i)]
        _, num, _ = used_data.shape 
        used_label = np.zeros(num,) + clip_label[ind+10]
        used_data = np.swapaxes(used_data, 0, 1)
        if 'all' in frequencies_selection:
            used_data = np.reshape(used_data, (num, -1))
        else:
            # apenas uma frequencia por vez
            # TODO processamento paralelo de cada frequencia
            frequencie_idx = all_freqs_dict[frequencies_selection[0]]
            used_data = used_data[:,:,frequencie_idx]
        used_data = np.reshape(used_data, (num, -1))
        print(f"\t\t{feature_name + str(i)} - video {str(i)} - {used_data.shape} observacoes")
        test_data = np.vstack((test_data, used_data))
        test_label = np.hstack((test_label, used_label))
    return train_data, test_data, train_label, test_label


In [87]:
train_data, test_data, train_label, test_label = generating_data(all_data, label_list, fn, frequencies_selection=['beta'])


	### TRAIN ###
## select only ['beta'] freq - idx: 3 ##
		de_LDS1 - video 1 - (235, 62) observacoes
		de_LDS2 - video 2 - (233, 62) observacoes
		de_LDS3 - video 3 - (206, 62) observacoes
		de_LDS4 - video 4 - (238, 62) observacoes
		de_LDS5 - video 5 - (185, 62) observacoes
		de_LDS6 - video 6 - (195, 62) observacoes
		de_LDS7 - video 7 - (237, 62) observacoes
		de_LDS8 - video 8 - (216, 62) observacoes
		de_LDS9 - video 9 - (265, 62) observacoes
	### TEST ###
		de_LDS10 - video 10 - (237, 62) observacoes
		de_LDS11 - video 11 - (235, 62) observacoes
		de_LDS12 - video 12 - (233, 62) observacoes
		de_LDS13 - video 13 - (235, 62) observacoes
		de_LDS14 - video 14 - (238, 62) observacoes
		de_LDS15 - video 15 - (206, 62) observacoes
